In [1]:
import rospy
import tf2_ros
from geometry_msgs.msg import PoseStamped, Point, Quaternion, Vector3Stamped, PointStamped, Vector3
from giskard_msgs.msg import WorldBody, CollisionEntry, WorldGoal, GiskardError
from giskardpy.python_interface.python_interface import GiskardWrapper
import math
from bayes_opt import BayesianOptimization
from giskardpy.motion_graph.tasks.task import WEIGHT_BELOW_CA


def reset_giskard():
    giskard.clear_motion_goals_and_monitors()
    giskard.world.clear()
    default_pose = {
        'r_elbow_flex_joint': -0.15,
        'r_forearm_roll_joint': 0,
        'r_shoulder_lift_joint': 0,
        'r_shoulder_pan_joint': 0,
        'r_upper_arm_roll_joint': 0,
        'r_wrist_flex_joint': -0.10001,
        'r_wrist_roll_joint': 0,
        'l_elbow_flex_joint': -0.15,
        'l_forearm_roll_joint': 0,
        'l_shoulder_lift_joint': 0,
        'l_shoulder_pan_joint': 0,
        'l_upper_arm_roll_joint': 0,
        'l_wrist_flex_joint': -0.10001,
        'l_wrist_roll_joint': 0,
        'torso_lift_joint': 0.2,
        'head_pan_joint': 0,
        'head_tilt_joint': 0,
        'l_gripper_l_finger_joint': 0.55,
        'r_gripper_l_finger_joint': 0.55
    }

    done = giskard.monitors.add_set_seed_configuration(default_pose)
    base_pose = PoseStamped()
    base_pose.header.frame_id = 'map'
    base_pose.pose.position = Point(0, 0, 0)
    base_pose.pose.orientation.w = 1
    done2 = giskard.monitors.add_set_seed_odometry(base_pose=base_pose)
    giskard.motion_goals.allow_all_collisions()
    giskard.monitors.add_end_motion(start_condition=f'{done} and {done2}')
    giskard.execute()
    giskard.clear_motion_goals_and_monitors()

In [2]:
rospy.init_node('giskard_examples')

In [3]:
giskard = GiskardWrapper()

In [5]:
reset_giskard()

# variables
single_joint_goal = {'torso_lift_joint': 0.3}
tool_frame = 'l_gripper_tool_frame'
cam_frame = 'wide_stereo_optical_frame'
base_link = 'base_footprint'
gripper_joint = 'l_gripper_l_finger_joint'
gripper_joint_open = 0.55
gripper_joint_close = 0.1
gripper_link = tool_frame


def giskard_sample(p1, p2):
    reset_giskard()
    # adding the object
    cyl_pose = PoseStamped()
    cyl_pose.header.frame_id = 'map'
    cyl_pose.pose.orientation.w = 1
    cyl_pose.pose.position = Point(2, -0.2, 1.2)
    giskard.world.add_cylinder(name='mycyl', height=0.25, radius=0.02, pose=cyl_pose, parent_link='map')

    obstacle_pose = PoseStamped()
    obstacle_pose.header.frame_id = 'mycyl'
    obstacle_pose.pose.orientation.w = 1
    obstacle_pose.pose.position = Point(-0.1, 0.1, 0)
    # giskard.world.add_box(name='obstacle', size=(0.02, 0.2, 0.1), pose=obstacle_pose, parent_link='mycyl')

    # define the robot features
    robot_pointing_feature = Vector3Stamped()
    robot_pointing_feature.header.frame_id = gripper_link
    robot_pointing_feature.vector = Vector3(1, 0, 0)

    robot_point_feature = PointStamped()
    robot_point_feature.header.frame_id = gripper_link
    robot_point_feature.point = Point(0, 0, 0)

    robot_gripper_z_axis_feature = Vector3Stamped()
    robot_gripper_z_axis_feature.header.frame_id = gripper_link
    robot_gripper_z_axis_feature.vector = Vector3(0, 0, 1)

    # define the world features
    world_cylinder_center_feature = PointStamped()
    world_cylinder_center_feature.header.frame_id = 'mycyl'
    world_cylinder_center_feature.point = Point(0, 0, 0)

    world_cyl_z_axis_feature = Vector3Stamped()
    world_cyl_z_axis_feature.header.frame_id = 'mycyl'
    world_cyl_z_axis_feature.vector = Vector3(0, 0, 1)

    # define the constraints
    mon_dist = giskard.monitors.add_distance(root_link='map',
                                             tip_link=gripper_link,
                                             reference_point=world_cylinder_center_feature,
                                             tip_point=robot_point_feature,
                                             lower_limit=0.01 - 0.005,
                                             upper_limit=0.01 + 0.005)
    giskard.motion_goals.add_distance(root_link='map',
                                      tip_link=gripper_link,
                                      reference_point=world_cylinder_center_feature,
                                      tip_point=robot_point_feature,
                                      lower_limit=0.01,
                                      upper_limit=0.01,
                                      end_condition=mon_dist)

    mon_height = giskard.monitors.add_height(root_link='map',
                                             tip_link=gripper_link,
                                             reference_point=world_cylinder_center_feature,
                                             tip_point=robot_point_feature,
                                             lower_limit=-0.09,
                                             upper_limit=0.09)
    giskard.motion_goals.add_height(root_link='map',
                                    tip_link=gripper_link,
                                    reference_point=world_cylinder_center_feature,
                                    tip_point=robot_point_feature,
                                    lower_limit=p1,
                                    upper_limit=p2,
                                    end_condition=mon_height)

    mon_pointing = giskard.monitors.add_pointing_at(root_link='map',
                                                    tip_link=gripper_link,
                                                    goal_point=world_cylinder_center_feature,
                                                    pointing_axis=robot_pointing_feature)
    giskard.motion_goals.add_pointing(root_link='map',
                                      tip_link=gripper_link,
                                      goal_point=world_cylinder_center_feature,
                                      pointing_axis=robot_pointing_feature,
                                      end_condition=mon_pointing)

    mon_align = giskard.monitors.add_vectors_aligned(root_link='map',
                                                     tip_link=gripper_link,
                                                     goal_normal=world_cyl_z_axis_feature,
                                                     tip_normal=robot_gripper_z_axis_feature)
    giskard.motion_goals.add_align_planes(root_link='map',
                                          tip_link=gripper_link,
                                          goal_normal=world_cyl_z_axis_feature,
                                          tip_normal=robot_gripper_z_axis_feature,
                                          end_condition=mon_align)

    # giskard.motion_goals.allow_all_collisions()
    # giskard.motion_goals.avoid_collision(min_distance=p3, group1='obstacle')
    giskard.monitors.add_end_motion(start_condition=f'{mon_dist} and {mon_height} and {mon_align} and {mon_pointing}')
    giskard.monitors.add_cancel_motion(giskard.monitors.add_local_minimum_reached(),
                                       error_message='local minimum reached while monitors are not satisfied')
    result = giskard.execute()
    if result.error.code != GiskardError.SUCCESS:
        print(giskard.get_end_motion_reason(move_result=result, show_all=False))
        # tf_buffer = tf2_ros.Buffer()
        # tf_listener = tf2_ros.TransformListener(tf_buffer)
        # transform = tf_buffer.lookup_transform('l_gripper_tool_frame', 'mycyl', rospy.Time(0), rospy.Duration(1.0))
        # return -math.sqrt(transform.transform.translation.x**2 + transform.transform.translation.y**2)
        return 0
    else:
        return 1



In [93]:
# Bounded region of parameter space
pbounds = {'p1': (-0.08, 0), 'p2': (0, 0.08)}

optimizer = BayesianOptimization(
    f=giskard_sample,
    pbounds=pbounds,
    random_state=378,
)

In [94]:
optimizer.maximize(
    init_points=3,
    n_iter=3,
)

|   iter    |  target   |    p1     |    p2     |    p3     |
-------------------------------------------------------------
{'PointingAt': False}
| 1         | 0.0       | -0.03245  | 0.01357   | 0.01451   |
{'PointingAt': False}
| 2         | 0.0       | -0.01469  | 0.03815   | 0.01358   |
{'PointingAt': False}
| 3         | 0.0       | -0.07842  | 0.01674   | 0.07861   |
| 4         | 1.0       | -0.000687 | 0.07616   | 0.09925   |
{'PointingAt': False}
| 5         | 0.0       | -0.06557  | 0.05318   | 0.08829   |
{'PointingAt': False}
| 6         | 0.0       | -0.05707  | 0.07648   | 0.01368   |


In [62]:
print(optimizer.max)

{'target': -0.0007272502698663801, 'params': {'p1': -0.09998856251826552, 'p2': 0.030233257263183978}}


In [7]:
giskard_sample(-0.05, 0.05)

1

In [ ]:
%%bash
pip3 install bayesian_optimization==1.4.0
pip3 install colorama==0.4.4

In [8]:
giskard.world.clear()
base_pose = PoseStamped()
base_pose.header.frame_id = 'map'
base_pose.pose.position = Point(0, 0, 0)
base_pose.pose.orientation.w = 1
done2 = giskard.monitors.add_set_seed_odometry(base_pose=base_pose)
giskard.motion_goals.allow_all_collisions()
giskard.monitors.add_end_motion(start_condition=f'{done2}')
giskard.execute()
giskard.clear_motion_goals_and_monitors()


bowl_pose = PoseStamped()
bowl_pose.header.frame_id = 'map'
bowl_pose.pose.orientation.w = 1
bowl_pose.pose.position = Point(1, 0, 0.5)  # Set bowl position (e.g., on a table)
giskard.world.add_cylinder(name='bowl', height=0.1, radius=0.15, pose=bowl_pose, parent_link='map')

# Define cup pose (initially above the bowl)
# cup_pose = PoseStamped()
# cup_pose.header.frame_id = 'map'
# cup_pose.pose.orientation.w = 1
# cup_pose.pose.position = Point(0, 0, 0.7)  # Set the cup 20 cm above the bowl
# giskard.world.add_cylinder(name='cup', height=0.15, radius=0.05, pose=cup_pose, parent_link='map')

# Define the robot's cup feature (position and orientation)
robot_cup_feature = PointStamped()
robot_cup_feature.header.frame_id = 'hand_palm_link'
robot_cup_feature.point = Point(0, 0, 0)

robot_gripper_z_axis_feature = Vector3Stamped()
robot_gripper_z_axis_feature.header.frame_id = 'hand_palm_link'
robot_gripper_z_axis_feature.vector = Vector3(1, 0, 0)

# Define the world features (center and z-axis of the bowl)
world_bowl_center_feature = PointStamped()
world_bowl_center_feature.header.frame_id = 'bowl'
world_bowl_center_feature.point = Point(0, 0, 0)

world_bowl_z_axis_feature = Vector3Stamped()
world_bowl_z_axis_feature.header.frame_id = 'bowl'
world_bowl_z_axis_feature.vector = Vector3(0, 0, 1)

# Maintain height between the cup and the bowl (along the z-axis)
mon_initial_height = giskard.monitors.add_height(
    root_link='map',
    tip_link='hand_palm_link',
    reference_point=world_bowl_center_feature,
    tip_point=robot_cup_feature,
    lower_limit=0.1,  # Maintain 10 cm above the bowl initially
    upper_limit=0.1 + 0.01,
    name='height1'
)
giskard.motion_goals.add_height(
    root_link='map',
    tip_link='hand_palm_link',
    reference_point=world_bowl_center_feature,
    tip_point=robot_cup_feature,
    lower_limit=0.1,
    upper_limit=0.1,
    end_condition=mon_initial_height,
    name='height1'
)

# Maintain distance between the cup and the bowl in the x-y plane
mon_distance = giskard.monitors.add_distance(
    root_link='map',
    tip_link='hand_palm_link',
    reference_point=world_bowl_center_feature,
    tip_point=robot_cup_feature,
    lower_limit=0,  # Align cup exactly over the bowl horizontally
    upper_limit=0 + 0.01
)
giskard.motion_goals.add_distance(
    root_link='map',
    tip_link='hand_palm_link',
    reference_point=world_bowl_center_feature,
    tip_point=robot_cup_feature,
    lower_limit=0,
    upper_limit=0,
    end_condition=mon_distance
)

# Align the cup's z-axis with the bowl's z-axis for pouring
mon_align = giskard.monitors.add_vectors_aligned(
    root_link='map',
    tip_link='hand_palm_link',
    goal_normal=world_bowl_z_axis_feature,
    tip_normal=robot_gripper_z_axis_feature
)
giskard.motion_goals.add_align_planes(
    root_link='map',
    tip_link='hand_palm_link',
    goal_normal=world_bowl_z_axis_feature,
    tip_normal=robot_gripper_z_axis_feature,
    end_condition=mon_align
)

# --- New Motion Goals to Raise Cup ---
# Monitor to check if the new height (20 cm above the bowl) is met
mon_new_height = giskard.monitors.add_height(
    root_link='map',
    tip_link='hand_palm_link',
    reference_point=world_bowl_center_feature,
    tip_point=robot_cup_feature,
    lower_limit=0.2,  # Desired new height 20 cm
    upper_limit=0.2 + 0.01,
    name='height2',
    start_condition=f'{mon_initial_height} and {mon_distance} and {mon_align}'
)
giskard.motion_goals.add_height(
    root_link='map',
    name='height2',
    tip_link='hand_palm_link',
    reference_point=world_bowl_center_feature,
    tip_point=robot_cup_feature,
    lower_limit=0.2,
    upper_limit=0.2,
    start_condition=f'{mon_initial_height} and {mon_distance} and {mon_align}',
    # Start when initial conditions are met
    end_condition=mon_new_height  # End motion when the cup is 20 cm above the bowl
)

# Monitor and execute the motion
giskard.monitors.add_end_motion(
    start_condition=mon_new_height  # End motion when the new height is reached
)
giskard.monitors.add_cancel_motion(
    giskard.monitors.add_local_minimum_reached(),
    error_message='local minimum reached while monitors are not satisfied'
)

giskard.motion_goals.allow_all_collisions()
# Execute the motion
result = giskard.execute()
if result.error.code != GiskardError.SUCCESS:
    print(giskard.get_end_motion_reason(move_result=result, show_all=False))
else:
    print("Cup successfully positioned above the bowl for pouring.")

Cup successfully positioned above the bowl for pouring.


In [25]:
giskard.world.clear()
base_pose = PoseStamped()
base_pose.header.frame_id = 'map'
base_pose.pose.position = Point(0, 0, 0)
base_pose.pose.orientation.w = 1
done2 = giskard.monitors.add_set_seed_odometry(base_pose=base_pose)
giskard.motion_goals.allow_all_collisions()
giskard.monitors.add_end_motion(start_condition=f'{done2}')
giskard.execute()
giskard.clear_motion_goals_and_monitors()



# Define variables for lower and upper limits for easy configuration
initial_cup_distance_x = 0.1  # Distance from the cup along the x-axis (in front of the cup)
initial_cup_distance_y = 0.0  # Align horizontally along the y-axis
initial_cup_height = 0.4  # Height of the cup

bowl_height = 0.5  # Height of the bowl (e.g., sitting on a table)
above_bowl_height = 0.2  # Desired height above the bowl to position the gripper

# --- Phase 1: Move the gripper in front of the new cup ---
# Define the pose of the new cup
cup_pose = PoseStamped()
cup_pose.header.frame_id = 'map'
cup_pose.pose.orientation.w = 1
cup_pose.pose.position = Point(1, -1, initial_cup_height)  # Position the cup at (0.5, 0, 0.15)
giskard.world.add_cylinder(name='cup', height=0.15, radius=0.05, pose=cup_pose, parent_link='map')

# Define the robot's hand palm link (gripper)
gripper_link = 'hand_palm_link'

# Define gripper's features (position and orientation for the 'hand_palm_link')
gripper_point_feature = PointStamped()
gripper_point_feature.header.frame_id = gripper_link
gripper_point_feature.point = Point(0, 0, 0)

gripper_z_axis_feature = Vector3Stamped()
gripper_z_axis_feature.header.frame_id = gripper_link
gripper_z_axis_feature.vector = Vector3(1, 0, 0)

# Define the world features (center and z-axis of the cup)
cup_center_feature = PointStamped()
cup_center_feature.header.frame_id = 'cup'
cup_center_feature.point = Point(0, 0, 0)

cup_z_axis_feature = Vector3Stamped()
cup_z_axis_feature.header.frame_id = 'cup'
cup_z_axis_feature.vector = Vector3(0, 0, 1)

# --- Motion Goals to Move the Gripper in Front of the Cup ---
# Move gripper in front of the cup (along the x-axis by a certain distance)
mon_gripper_to_cup = giskard.monitors.add_distance(
    name="mon_gripper_to_cup",
    root_link='map',
    tip_link=gripper_link,
    reference_point=cup_center_feature,
    tip_point=gripper_point_feature,
    lower_limit=initial_cup_distance_x,  # Distance in front of the cup along the x-axis
    upper_limit=initial_cup_distance_x+0.01
)

# Align the gripper's z-axis with the cup's z-axis
mon_align_gripper_with_cup = giskard.monitors.add_vectors_aligned(
    name="mon_align_gripper_with_cup",
    root_link='map',
    tip_link=gripper_link,
    goal_normal=cup_z_axis_feature,
    tip_normal=gripper_z_axis_feature
)

# Motion goal: Move in front of the cup, ending when all monitors in this phase are satisfied
giskard.motion_goals.add_distance(
    name="goal_gripper_to_cup",
    root_link='map',
    tip_link=gripper_link,
    reference_point=cup_center_feature,
    tip_point=gripper_point_feature,
    lower_limit=initial_cup_distance_x,
    upper_limit=initial_cup_distance_x,
    end_condition=f'{mon_gripper_to_cup} and {mon_align_gripper_with_cup}'  # End only when both monitors are satisfied
)

giskard.motion_goals.add_align_planes(
    name="goal_align_gripper_with_cup",
    root_link='map',
    tip_link=gripper_link,
    goal_normal=cup_z_axis_feature,
    tip_normal=gripper_z_axis_feature,
    end_condition=f'{mon_gripper_to_cup} and {mon_align_gripper_with_cup}'  # Same condition for both goals
)

# --- Phase 2: Move the Gripper Above the Bowl ---
# Define the pose of the bowl
bowl_pose = PoseStamped()
bowl_pose.header.frame_id = 'map'
bowl_pose.pose.orientation.w = 1
bowl_pose.pose.position = Point(1, 0, bowl_height)  # Bowl position at (0, 0, 0.5)
giskard.world.add_cylinder(name='bowl', height=0.1, radius=0.15, pose=bowl_pose, parent_link='map')

# Define world features (center and z-axis of the bowl)
bowl_center_feature = PointStamped()
bowl_center_feature.header.frame_id = 'bowl'
bowl_center_feature.point = Point(0, 0, 0)

bowl_z_axis_feature = Vector3Stamped()
bowl_z_axis_feature.header.frame_id = 'bowl'
bowl_z_axis_feature.vector = Vector3(0, 0, 1)

# Move the gripper directly above the bowl
mon_gripper_above_bowl = giskard.monitors.add_distance(
    name="mon_gripper_above_bowl",
    root_link='map',
    tip_link=gripper_link,
    reference_point=bowl_center_feature,
    tip_point=gripper_point_feature,
    lower_limit=0,  # Keep the gripper exactly above the bowl horizontally
    upper_limit=0+0.01,
    start_condition=f'{mon_gripper_to_cup} and {mon_align_gripper_with_cup}'  # Start after moving in front of the cup
)

# Move the gripper to a height above the bowl
mon_gripper_height_above_bowl = giskard.monitors.add_height(
    name="mon_gripper_height_above_bowl",
    root_link='map',
    tip_link=gripper_link,
    reference_point=bowl_center_feature,
    tip_point=gripper_point_feature,
    lower_limit=above_bowl_height,  # Desired height above the bowl (20 cm)
    upper_limit=above_bowl_height+0.01,
    start_condition=f'{mon_gripper_to_cup} and {mon_align_gripper_with_cup}'  # Start once gripper is above the bowl
)

# Align the gripper's z-axis with the bowl's z-axis for correct orientation
mon_align_gripper_with_bowl = giskard.monitors.add_vectors_aligned(
    name="mon_align_gripper_with_bowl",
    root_link='map',
    tip_link=gripper_link,
    goal_normal=bowl_z_axis_feature,
    tip_normal=gripper_z_axis_feature,
    start_condition=f'{mon_gripper_to_cup} and {mon_align_gripper_with_cup}'  # Start after gripper is horizontally aligned above bowl
)

# Motion goal: Move gripper above the bowl, ending when all monitors in this phase are satisfied
giskard.motion_goals.add_distance(
    name="goal_gripper_above_bowl",
    root_link='map',
    tip_link=gripper_link,
    reference_point=bowl_center_feature,
    tip_point=gripper_point_feature,
    lower_limit=0,
    upper_limit=0,
    start_condition=f'{mon_gripper_to_cup} and {mon_align_gripper_with_cup}',  # Trigger after cup is reached
    end_condition=f'{mon_gripper_above_bowl} and {mon_gripper_height_above_bowl} and {mon_align_gripper_with_bowl}'  # All monitors must be true
)

giskard.motion_goals.add_height(
    name="goal_gripper_height_above_bowl",
    root_link='map',
    tip_link=gripper_link,
    reference_point=bowl_center_feature,
    tip_point=gripper_point_feature,
    lower_limit=above_bowl_height,
    upper_limit=above_bowl_height,
    start_condition=f'{mon_gripper_to_cup} and {mon_align_gripper_with_cup}',  # Start once gripper is above bowl
    end_condition=f'{mon_gripper_above_bowl} and {mon_gripper_height_above_bowl} and {mon_align_gripper_with_bowl}'  # All monitors must be true
)

giskard.motion_goals.add_align_planes(
    name="goal_align_gripper_with_bowl",
    root_link='map',
    tip_link=gripper_link,
    goal_normal=bowl_z_axis_feature,
    tip_normal=gripper_z_axis_feature,
    start_condition=f'{mon_gripper_to_cup} and {mon_align_gripper_with_cup}',  # Start once gripper is above the bowl
    end_condition=f'{mon_gripper_above_bowl} and {mon_gripper_height_above_bowl} and {mon_align_gripper_with_bowl}'  # All monitors must be true
)
# End the motion once the new height above the bowl and alignment are achieved
giskard.monitors.add_end_motion(
    start_condition=f'{mon_gripper_height_above_bowl} and {mon_align_gripper_with_bowl} and {mon_gripper_above_bowl}'
)
giskard.monitors.add_cancel_motion(
    giskard.monitors.add_local_minimum_reached(),
    error_message='Local minimum reached while monitors are not satisfied'
)

# Execute the motion
giskard.motion_goals.allow_all_collisions()
result = giskard.execute()
if result.error.code != GiskardError.SUCCESS:
    print(giskard.get_end_motion_reason(move_result=result, show_all=False))
else:
    print("Gripper successfully moved in front of the cup and above the bowl.")


Gripper successfully moved in front of the cup and above the bowl.


In [23]:
import casadi as ca

def python_to_casadi(python_function, num_input):
    # Define CasADi symbols for the variables
    variables = ca.MX.sym('x', num_input)

    # Convert Python function to CasADi function
    casadi_function = python_function(variables)

    # Create CasADi function
    casadi_func = ca.Function('f', variables, [casadi_function])

    return casadi_func

# Define your mathematical function in Python
def my_function(x):
    return x**2

# Number of input variables
num_input = 1

# Convert the function
casadi_func = python_to_casadi(my_function, num_input)

# Evaluate the CasADi function
x_val = 2  # Input values as a CasADi column vector
print(casadi_func(x_val).full()[0])

x


NotImplementedError: Wrong number or type of arguments for overloaded function 'new_Function'.
  Possible prototypes are:
    Function()
    Function(str)
    Function(Function)
    Function(str,[SX],[SX],dict)
    Function(str,[MX],[MX],dict)
    Function(str,dict:SX,[str],[str],dict)
    Function(str,dict:MX,[str],[str],dict)
    Function(str,[SX],[SX],[str],[str],dict)
    Function(str,[MX],[MX],[str],[str],dict)
  You have: '(str,MX,[MX])'
